Installing necessary packages:
```Julia
using Pkg

Pkg.add(["Random", "Distributions", "DataFrames", "GLM", "Statistics", "ProgressMeter", "Plots", "Turing])
Pkg.add(url = "https://github.com/ncn-foreigners/UnobservedCountEstimation.jl")
```

In [1]:
using Turing

In [2]:
using Random, Distributions, DataFrames, GLM, Statistics, ProgressMeter, CSV, StatsBase, StatsPlots

In [12]:
α, β = rand(MvNormal([1.6, 2.4], .3 * [1 .5; .5 1]))
Q = 20
nsims = 10000

N_distr = Poisson(700)
M_distr = (N, α) -> Poisson.(N .^ α)
u_distr = (n, N) -> Beta(n, N - n)
#u_distr = (M, μ) -> Gamma(ϕ, 1 / ϕ)

res = Vector{Any}(missing, nsims)

prog = Progress(10*Threads.nthreads(), "Simulation progress ...")

Threads.@threads for i in 1:(5*Threads.nthreads())
    print(string(Threads.threadid()) * " ")
    next!(prog)
end # end for

1 4 3 2 2 3 4 2 4 3 4 1 2 3 2 4 2 3 4 4 

In [13]:
α, β

(0.9902382173890572, 1.7093018837754483)

Testing `Trung.jl`

In [14]:
N = rand.(N_distr, Q)
M = reduce(vcat, rand.(M_distr(N, α), 1))
n = reduce(vcat, rand.(Binomial.(N, .05), 1))

p = (N .^ α) .* ((n ./ N) .^ β)
p = 1 ./ (1 .+ exp.(-p))

u = 1.0
m = reduce(vcat, rand.(Binomial.(M, p * u), 1))

[m M n N p]

20×5 Matrix{Float64}:
 669.0  677.0  38.0  700.0  0.98916
 596.0  626.0  30.0  665.0  0.957972
 638.0  649.0  37.0  680.0  0.987911
 662.0  663.0  46.0  706.0  0.998012
 641.0  666.0  31.0  687.0  0.961957
 638.0  668.0  32.0  689.0  0.967806
 622.0  628.0  40.0  685.0  0.993337
 673.0  679.0  37.0  721.0  0.985539
 671.0  688.0  35.0  708.0  0.97996
 667.0  683.0  33.0  698.0  0.972179
 612.0  653.0  29.0  715.0  0.942709
 621.0  640.0  32.0  688.0  0.967917
 648.0  653.0  37.0  654.0  0.989318
 636.0  642.0  40.0  684.0  0.993372
 658.0  684.0  31.0  710.0  0.959092
 619.0  652.0  30.0  700.0  0.95317
 673.0  690.0  33.0  690.0  0.972968
 685.0  732.0  29.0  709.0  0.943621
 542.0  647.0  21.0  716.0  0.833608
 647.0  654.0  40.0  726.0  0.991835

In [16]:
@model function binomial_model(m, n, N)
    γ ~ MvNormal([1.6, 2.4], .3 * [1 .5; .5 1])
    ξ = N .^ γ[1]
    μ = ξ .* ((n ./ N) .^ γ[2])
    μ = 1 ./ (1 .+ exp.(-μ))
    M = Vector(undef, length(m))
    
    for i in eachindex(M)
        M[i] ~ Poisson(ξ[i])
    end

    for i in eachindex(m)
        m[i] ~ Binomial(M[i], μ[i])
    end
    return m
end

chains_1 = sample(binomial_model(m, n, N), PG(100), MCMCThreads(), 500, 4)

Chains MCMC chain (500×24×4 Array{Float64, 3}):

Iterations        = 1:1:500
Number of chains  = 4
Samples per chain = 500
Wall duration     = 203.04 seconds
Compute duration  = 557.78 seconds
parameters        = γ[1], γ[2], M[1], M[2], M[3], M[4], M[5], M[6], M[7], M[8], M[9], M[10], M[11], M[12], M[13], M[14], M[15], M[16], M[17], M[18], M[19], M[20]
internals         = lp, logevidence

Summary Statistics
  parameters        mean       std      mcse   ess_bulk   ess_tail      rhat   ⋯
      Symbol     Float64   Float64   Float64    Float64    Float64   Float64   ⋯

        γ[1]      1.0803    0.0098    0.0030     7.3911        NaN    1.9009   ⋯
        γ[2]      3.0257    0.2438    0.0880     9.9830    31.2019    1.8498   ⋯
        M[1]   1194.1720   75.5468   25.9927     6.8740        NaN    1.7161   ⋯
        M[2]   1117.9295   72.5972   22.6175     7.5991        NaN    1.5091   ⋯
        M[3]   1151.5975   70.0275   20.0260    11.5770    11.0687    1.3946   ⋯
        M[4]   1198.6

In [17]:
[α, β, (N .^ α)', M, describe(chains_1[1:end,:,:])[1][3:end, :mean]]

5-element Vector{Any}:
 0.9902382173890572
 1.7093018837754483
  [656.6361848299803 624.1168013773231 … 671.4968535286756 680.783120714251]
  [677, 626, 649, 663, 666, 668, 628, 679, 688, 683, 653, 640, 653, 642, 684, 652, 690, 732, 647, 654]
  [1194.172, 1117.9295, 1151.5975, 1198.652, 1166.812, 1160.572, 1151.273, 1225.556, 1233.0835, 1198.9535, 1217.853, 1161.73, 1115.2235, 1147.2875, 1204.8765, 1184.223, 1175.4345, 1200.4165, 1158.674, 1226.143]

In [18]:
@model function binomial_model(m, n, N, μ, Σ)
    γ ~ MvNormal(μ, Σ)
    ξ = N .^ γ[1]
    μ = ξ .* ((n ./ N) .^ γ[2])
    μ = 1 ./ (1 .+ exp.(-μ))
    M = Vector(undef, length(m))
    
    for i in eachindex(M)
        M[i] ~ Poisson(ξ[i])
    end

    for i in eachindex(m)
        m[i] ~ Binomial(M[i], μ[i])
    end
    return m
end
setprogress!(false)

[ Info: [Turing]: progress logging is disabled globally
[ Info: [AdvancedVI]: global PROGRESS is set as false


false

In [36]:
prog = Progress(nsims, "Simulation progress ...")
res = Vector{Any}(missing, nsims)
μ = [1.5, 3.4]
Σ = .2 * [1 -.6; -.6 1]
nsims = 100

Threads.@threads for k in 1:nsims
    α, β = rand(MvNormal(μ, Σ))

    N = rand.(N_distr, Q)
    M = reduce(vcat, rand.(M_distr(N, α), 1))
    n = reduce(vcat, rand.(Binomial.(N, .2), 1))

    p = (N .^ α) .* ((n ./ N) .^ β)
    p = 1 ./ (1 .+ exp.(-p))

    #u = reduce(vcat, rand.(u_distr.(n, N), 1))
    u = 1.0
    m = reduce(vcat, rand.(Binomial.(M, p * u), 1))

    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )

    mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    α̂₁, β̂₁ = coef(mm)

    ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    α̂₂, β̂₂ = coef(ols)

    est = sample(binomial_model(m, n, N, μ, Σ), PG(100), 500)
    α̂₃, β̂₃ = describe(est)[1][1:2, :mean]
    M̂ = describe(est)[1][3:end, :mean]

    μ̂_prior = [α̂₁, β̂₁]
    Σ̂_prior = vcov(mm)

    est_2 = sample(binomial_model(m, n, N, μ̂_prior, Σ̂_prior), PG(100), 500)
    α̂₄, β̂₄ = describe(est_2)[1][1:2, :mean]
    M̂_2 = describe(est_2)[1][3:end, :mean]

    res[k] = [
        sum(M); sum(N .^ α); sum(N .^ α̂₁); sum(N .^ α̂₂); sum(N .^ α̂₃);
        sum([mean(N[x] .^ est[:, 1, :]) for x in 1:Q]); sum(M̂); 
        sum(N .^ α̂₄); sum([mean(N[x] .^ est_2[:, 1, :]) for x in 1:Q]);
        sum(M̂_2)
    ]
    #push!(res, [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)])
    next!(prog)
end # end for

df_res = DataFrame(mapreduce(permutedims, vcat, res), 
    ["actual", "expected", "glm_est", "ols_est", "expected_est" , "expected_est_better", 
        "actual_est", "expected_est_bad_prior" , "expected_est_better_bad_prior", "actual_est_bad_prior"])

describe(df_res)

Simulation progress ... 100%|████████████████████████████| Time: 5:42:53


LoadError: ArgumentError: 'Vector{Float64}' iterates 'Float64' values, which doesn't satisfy the Tables.jl `AbstractRow` interface

In [47]:
df_res = DataFrame(mapreduce(permutedims, vcat, res), 
    ["actual", "expected", "glm_est", "ols_est", "expected_est" , "expected_est_better", 
        "actual_est", "expected_est_bad_prior" , "expected_est_better_bad_prior", "actual_est_bad_prior"])

describe(df_res)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,actual,1.57611e7,618.0,354958.0,9.37526e8,0,Float64
2,expected,1.57604e7,582.652,3.55792e5,9.37469e8,0,Float64
3,glm_est,1.57724e7,640.196,3.52512e5,9.37852e8,0,Float64
4,ols_est,1.57718e7,601.937,3.5178e5,9.37896e8,0,Float64
5,expected_est,2.72258e5,646.569,2.89695e5,9.24024e5,0,Float64
6,expected_est_better,1.19835e7,647.684,4.08845e5,1.61499e8,0,Float64
7,actual_est,1.19835e7,645.354,4.08984e5,1.615e8,0,Float64
8,expected_est_bad_prior,1.57714e7,500.085,3.54172e5,9.37835e8,0,Float64
9,expected_est_better_bad_prior,1.57716e7,500.913,3.54184e5,9.37835e8,0,Float64


In [51]:
DataFrame(
    rel_bias = [mean((df_res[:, 1] .- df_res[:, k]) ./ df_res[:, 1])        for k in 2:10],
    rel_mse  = [mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ df_res[:, 1]) for k in 2:10],
    rel_mae  = [mean(abs.(df_res[:, 1] .- df_res[:, k]) ./ df_res[:, 1])    for k in 2:10],
    est      = ["Expected value", "glm", "ols", "expected_est", "expected_est_better", "actual_est", 
        "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,0.000116574,1.20939,0.00437977,Expected value
2,-0.21841,9256.35,0.257574,glm
3,-0.215511,9065.91,0.26007,ols
4,0.281669,1.53741e7,0.377951,expected_est
5,-3.6523,1.50211e8,3.75249,expected_est_better
6,-3.65256,1.50211e8,3.75227,actual_est
7,-0.168579,8950.56,0.226103,expected_est_bad_prior
8,-0.175592,9447.54,0.232423,expected_est_better_bad_prior
9,-0.175543,9449.05,0.232135,actual_est_bad_prior


In [52]:
DataFrame(
    bias  = [mean((df_res[:, 1] .- df_res[:, k]) ./ 1000)        for k in 2:10],
    mse   = [mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000) for k in 2:10],
    rmse  = [sqrt(mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000)) for k in 2:10],
    var   = [var(df_res[:, k] / 1000) for k in 2:10],
    sd    = [std(df_res[:, k] / 1000) for k in 2:10],
    mae   = [mean(abs.(df_res[:, 1] .- df_res[:, k]) ./ 1000)    for k in 2:10],
    est   = ["Expected value", "glm", "ols", "expected_est", "expected_est_better", "actual_est", 
        "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"]
)

Row,bias,mse,rmse,var,sd,mae,est
,Float64,Float64,Float64,Float64,Float64,Float64,String
1,0.689031,37519.9,193.701,9.05793e9,95173.1,1.63098,Expected value
2,-11.3189,5.55979e6,2357.92,9.06598e9,95215.4,32.4284,glm
3,-10.6342,5.68851e6,2385.06,9.06663e9,95218.9,32.8607,ols
4,15488.9,9.20643e12,3.03421e6,44438.0,210.803,15502.9,expected_est
5,3777.58,9.01605e12,3.00267e6,5.25868e8,22931.8,20322.4,expected_est_better
6,3777.57,9.01606e12,3.00267e6,5.25871e8,22931.9,20322.5,actual_est
7,-10.2836,4.19614e6,2048.45,9.06521e9,95211.4,22.2577,expected_est_bad_prior
8,-10.4429,4.20785e6,2051.3,9.06521e9,95211.4,22.3761,expected_est_better_bad_prior
9,-10.5914,4.20476e6,2050.55,9.06516e9,95211.2,22.2999,actual_est_bad_prior


In [50]:
CSV.write(pwd() * "/data_raw/init_res.csv", df_res);

### Uknown prior

In [53]:
Q = 7 # Liczba państw

N_distr = Poisson(1500)

res = Vector{Any}(missing, nsims)
μ = [1.5, 3.4]
Σ = .2 * [1 -.6; -.6 1]
nsims = 100

prog = Progress(nsims, "Simulation progress ...")

Threads.@threads for k in 1:nsims
    α, β = rand(MvNormal(μ, Σ))

    N = rand.(N_distr, Q)
    M = reduce(vcat, rand.(M_distr(N, α), 1))
    n = reduce(vcat, rand.(Binomial.(N, .2), 1))

    p = (N .^ α) .* ((n ./ N) .^ β)
    p = 1 ./ (1 .+ exp.(-p))
    #u = reduce(vcat, rand.(u_distr.(n, N), 1))
    u = 1.0
    m = reduce(vcat, rand.(Binomial.(M, p * u), 1))
    
    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )
    
    # TODO:: estimate Σ via one of those
    mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    α̂₁, β̂₁ = coef(mm)

    ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    α̂₂, β̂₂ = coef(ols)

    est = sample(binomial_model(m, n, N, μ, Σ), PG(100), 500)
    α̂₃, β̂₃ = describe(est)[1][1:2, :mean]
    M̂ = describe(est)[1][3:end, :mean]

    μ̂_prior = [α̂₁, β̂₁]
    Σ̂_prior = vcov(mm)

    est_2 = sample(binomial_model(m, n, N, μ̂_prior, Σ̂_prior), PG(100), 500)
    α̂₄, β̂₄ = describe(est_2)[1][1:2, :mean]
    M̂_2 = describe(est_2)[1][3:end, :mean]

    res[k] = [
        sum(M); sum(N .^ α); sum(N .^ α̂₁); sum(N .^ α̂₂); sum(N .^ α̂₃);
        sum([mean(N[x] .^ est[:, 1, :]) for x in 1:Q]); sum(M̂); 
        sum(N .^ α̂₄); sum([mean(N[x] .^ est_2[:, 1, :]) for x in 1:Q]);
        sum(M̂_2)
    ]

    #push!(res, [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)])
    next!(prog)
end # end for

df_res = DataFrame(mapreduce(permutedims, vcat, res), 
    ["actual", "expected", "glm_est", "ols_est", "expected_est" , "expected_est_better", 
        "actual_est", "expected_est_bad_prior" , "expected_est_better_bad_prior", "actual_est_bad_prior"])

describe(df_res)

Simulation progress ... 100%|████████████████████████████| Time: 4:51:54


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,actual,2.15606e7,371.0,528770.0,1.06463e9,0,Float64
2,expected,2.15609e7,355.536,5.27528e5,1.06465e9,0,Float64
3,glm_est,2.15765e7,97.6248,5.09981e5,1.06657e9,0,Float64
4,ols_est,2.15759e7,94.0068,5.09053e5,1.06654e9,0,Float64
5,expected_est,2.54975e5,374.828,3.26216e5,512301.0,0,Float64
6,expected_est_better,3.93233e7,375.473,2.45891e7,3.45985e8,0,Float64
7,actual_est,3.93233e7,383.914,2.4589e7,3.45986e8,0,Float64
8,expected_est_bad_prior,2.15778e7,202.92,5.21789e5,1.06665e9,0,Float64
9,expected_est_better_bad_prior,2.15786e7,216.602,5.222e5,1.06665e9,0,Float64


In [54]:
DataFrame(
    rel_bias = [mean((df_res[:, 1] .- df_res[:, k]) ./ df_res[:, 1])        for k in 2:10],
    rel_mse  = [mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ df_res[:, 1]) for k in 2:10],
    rel_mae  = [mean(abs.(df_res[:, 1] .- df_res[:, k]) ./ df_res[:, 1])    for k in 2:10],
    est      = ["Expected value", "glm", "ols", "expected_est", "expected_est_better", "actual_est", 
        "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,0.00162797,1.06888,0.00396007,Expected value
2,-0.268251,6638.04,0.337673,glm
3,-0.257848,6271.36,0.327008,ols
4,0.376868,2.11475e7,0.4379,expected_est
5,-18.6108,1.30055e9,18.6637,expected_est_better
6,-18.6118,1.30056e9,18.6642,actual_est
7,-0.0393317,2399.75,0.131305,expected_est_bad_prior
8,-0.095784,4538.62,0.1828,expected_est_better_bad_prior
9,-0.0953988,4533.34,0.183023,actual_est_bad_prior


In [55]:
DataFrame(
    bias  = [mean((df_res[:, 1] .- df_res[:, k]) ./ 1000)        for k in 2:10],
    mse   = [mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000) for k in 2:10],
    rmse  = [sqrt(mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000)) for k in 2:10],
    var   = [var(df_res[:, k] / 1000) for k in 2:10],
    sd    = [std(df_res[:, k] / 1000) for k in 2:10],
    mae   = [mean(abs.(df_res[:, 1] .- df_res[:, k]) ./ 1000)    for k in 2:10],
    est   = ["Expected value", "glm", "ols", "expected_est", "expected_est_better", "actual_est", 
        "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"]
)

Row,bias,mse,rmse,var,sd,mae,est
,Float64,Float64,Float64,Float64,Float64,Float64,String
1,-0.269928,19961.3,141.285,1.26127e10,1.12306e5,1.69467,Expected value
2,-15.8847,5.35358e7,7316.82,1.26532e10,1.12486e5,71.8523,glm
3,-15.2771,5.20521e7,7214.71,1.26525e10,1.12483e5,71.4522,ols
4,21305.7,1.29334e13,3.5963e6,31108.9,176.377,21313.9,expected_est
5,-17762.7,1.05269e13,3.24452e6,3.65892e9,60489.0,40440.0,expected_est_better
6,-17762.7,1.05269e13,3.24452e6,3.65892e9,60489.0,40440.0,actual_est
7,-17.202,5.65178e7,7517.84,1.26551e10,1.12495e5,63.4928,expected_est_bad_prior
8,-17.9886,5.65988e7,7523.22,1.26551e10,1.12495e5,63.8227,expected_est_better_bad_prior
9,-18.0934,5.65669e7,7521.1,1.26551e10,1.12495e5,63.8077,actual_est_bad_prior


In [56]:
CSV.write(pwd() * "/data_raw/init_res_1.csv", df_res);